In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import json
cwd = os.getcwd()

In [2]:
bestiaryFiles = os.listdir("./bestiary")
spellFiles = os.listdir("./spells")

In [3]:
def has_dice(tag):
    return tag.has_attr("data-packed-dice")

In [4]:
conditions = ['blinded','charmed','deafened','frightened','grappled','incapacitated','invisible']
conditions += 'paralyzed','petrified','poisoned','prone','restrained','stunned','exhaustion'
conditionsRegex = ""
for i, condition in enumerate(conditions):
    condition = "[" + condition[0].upper() + condition[0].lower() + "]" + condition[1:]
    conditionsRegex += condition
    if(i<len(conditions)-1):
        conditionsRegex += "|"
conditionsRegex = re.compile(conditionsRegex)

casters = 0

nums_dict = {
    'one': '1',
    'two': '2',
    'three': '3',
    'four': '4',
    'five': '5',
    'six': '6',
    'seven': '7',
    'eight': '8',
    'nine': '9',
    'zero': '0'
}

for creatureFile in bestiaryFiles:
    
    size = ""
    armorClass = 0
    hitPoints = 0
    speed = {"walk":0,
             "climb":0,
             "fly":0,
             "swim":0,
             "burrow":0}
    
    stats = {"strength":0,
             "dexterity":0,
             "constitution":0,
             "intelligence":0,
             "wisdom":0,
             "charisma":0}
    
    savingThrows = {"strength":0,
                    "dexterity":0,
                    "constitution":0,
                    "intelligence":0,
                    "wisdom":0,
                    "charisma":0}
    
    skills = {"Athletics":0,
              "Acrobatics":0,
              "Sleight of Hand,":0,
              "Stealth":0,
              "Arcana":0,
              "History":0,
              "Investigation":0,
              "Nature":0,
              "Religion":0,
              "Animal Handling":0,
              "Insight":0,
              "Medicine":0,
              "Perception":0,
              "Survival":0,
              "Deception":0,
              "Intimidation":0,
              "Performance":0,
              "Persuasion":0,
             }
    
    damageResistances = {"Acid":False,
                         "Bludgeoning":False,
                         "Cold":False,
                         "Fire":False,
                         "Force":False,
                         "Lightning":False,
                         "Necrotic":False,
                         "Piercing":False,
                         "Poison":False,
                         "Psychic":False,
                         "Radiant":False,
                         "Slashing":False,
                         "Thunder":False,
    }
    damageVulnerabilities = {"Acid":False,
                         "Bludgeoning":False,
                         "Cold":False,
                         "Fire":False,
                         "Force":False,
                         "Lightning":False,
                         "Necrotic":False,
                         "Piercing":False,
                         "Poison":False,
                         "Psychic":False,
                         "Radiant":False,
                         "Slashing":False,
                         "Thunder":False,
    }
    damageImmunities = {"Acid":False,
                         "Bludgeoning":False,
                         "Cold":False,
                         "Fire":False,
                         "Force":False,
                         "Lightning":False,
                         "Necrotic":False,
                         "Piercing":False,
                         "Poison":False,
                         "Psychic":False,
                         "Radiant":False,
                         "Slashing":False,
                         "Thunder":False,
    }
    
    conditionImmunities = {"blinded":False,
                           "charmed":False,
                           "deafened":False,
                           "exhaustion":False,
                           "frightened":False,
                           "grappled":False,
                           "incapacitated":False,
                           "invisible":False,
                           "paralyzed":False,
                           "petrified":False,
                           "poisoned":False,
                           "prone":False,
                           "restrained":False,
                           "stunned":False,
                           "unconscious":False,
    }
    
    challengeRating = -1
    proficiencyBonus = -1
    
    actions = []
    bonuses = []
    reactions = []
    features = []

    legendaryAct = 0
    legendaryRes = 0
    mythic = False
    regional = False
    lair = False
    bonus = False
    reaction = False
    action = False
    multiAttack = False
    spellCaster = False
    with open(cwd + "\\bestiary\\" + creatureFile, "r") as f:
        soup = BeautifulSoup(f,'html.parser')
        name = soup.find_all("h1", {"class": "stats-name"})[0].string
        placesOfInterest = soup.find_all("td", {"class": "mon__sect-row-inner"})
        headersOfInterest = soup.find_all("h3", {"class":"mon__sect-header-inner"})
        
        data = []
        rows = soup.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele])
            
        armorClass = ''.join(filter(lambda i: i.isdigit(), data[4][0]))[-2:]
        hitPoints = re.search(r'\d+', data[5][0]).group()
        
        speed["walk"] = re.search(r'\d+', data[6][0]).group()
        if "fly" in data[6][0]:
            speed["fly"] = re.search(r'\d+', data[6][0].split("fly",1)[1]).group()
        if "climb" in data[6][0]:
            speed["climb"] = re.search(r'\d+', data[6][0].split("climb",1)[1]).group()
        if "swim" in data[6][0]:
            speed["swim"] = re.search(r'\d+', data[6][0].split("swim",1)[1]).group()
        if "burrow" in data[6][0]:
            speed["burrow"] = re.search(r'\d+', data[6][0].split("burrow",1)[1]).group()
            
        stats["strength"],savingThrows["strength"] = re.search(r'\d+', data[9][0]).group(), (int(re.search(r'\d+', data[9][0]).group())-10)//2
        skills["Athletics"] = savingThrows["strength"]
        stats["dexterity"], savingThrows["dexterity"] = re.search(r'\d+', data[9][1]).group(), (int(re.search(r'\d+', data[9][1]).group())-10)//2
        skills["Acrobatics"],skills["Sleight of Hand"], skills["Stealth"] = savingThrows["dexterity"],savingThrows["dexterity"],savingThrows["dexterity"]
        stats["constitution"], savingThrows["constitution"] = re.search(r'\d+', data[9][2]).group(), (int(re.search(r'\d+', data[9][2]).group())-10)//2
        stats["intelligence"], savingThrows["intelligence"] = re.search(r'\d+', data[9][3]).group(), (int(re.search(r'\d+', data[9][3]).group())-10)//2
        skills["Arcana"],skills["History"],skills["Investigation"],skills["Nature"],skills["Religion"] = savingThrows["intelligence"], savingThrows["intelligence"], savingThrows["intelligence"], savingThrows["intelligence"], savingThrows["intelligence"]
        stats["wisdom"], savingThrows["wisdom"] = re.search(r'\d+', data[9][4]).group(), (int(re.search(r'\d+', data[9][4]).group())-10)//2
        skills["Animal Handling"],skills["Insight"],skills["Medicine"],skills["Perception"],skills["Survival"] = savingThrows["wisdom"], savingThrows["wisdom"], savingThrows["wisdom"], savingThrows["wisdom"], savingThrows["wisdom"]
        stats["charisma"], savingThrows["charisma"] = re.search(r'\d+', data[9][5]).group(), (int(re.search(r'\d+', data[9][5]).group())-10)//2
        skills["Deception"],skills["Intimidation"],skills["Performance"],skills["Persuasion"] = savingThrows["charisma"], savingThrows["charisma"], savingThrows["charisma"], savingThrows["charisma"]
            
        if "Str" in data[11][0]:
            savingThrows["strength"] = re.search(r'\d+', data[11][0].split("Str",1)[1]).group()
        if "Dex" in data[11][0]:
            savingThrows["dexterity"] = re.search(r'\d+', data[11][0].split("Dex",1)[1]).group()
        if "Con" in data[11][0]:
            savingThrows["constitution"] = re.search(r'\d+', data[11][0].split("Con",1)[1]).group()
        if "Int" in data[11][0]:
            savingThrows["intelligence"] = re.search(r'\d+', data[11][0].split("Int",1)[1]).group()
        if "Wis" in data[11][0]:
            savingThrows["wisdom"] = re.search(r'\d+', data[11][0].split("Wis",1)[1]).group()
        if "Cha" in data[11][0]:
            savingThrows["charisma"] = re.search(r'\d+', data[11][0].split("Cha",1)[1]).group()
        
        
        for i in data:
            if(len(i) == 2):
                if "Challenge\\n" in i[0]:
                    challengeRating = re.search(r'\d+[\/]?\d?', i[0].split("Challenge\\n",1)[1]).group() 

                if "Proficiency Bonus" in i[1]:
                    proficiencyBonus = re.search(r'\d+', i[1].split("Proficiency Bonus",1)[1]).group()
            if i:
                if "Skills" in i[0]:
                    for k in skills:
                        if k in i[0]:
                            skills[k] = re.search(r'\d+', i[0].split(k,1)[1]).group()
                if "Damage Resistances" in i[0]:
                    for k in damageResistances:
                        if k.lower() in i[0]:
                            damageResistances[k] = True
                if "Damage Vulnerabilities" in i[0]:
                    for k in damageVulnerabilities:
                        if k.lower() in i[0]:
                            damageVulnerabilities[k] = True
                if "Damage Immunities" in i[0]:
                    for k in damageImmunities:
                        if k.lower() in i[0]:
                            damageImmunities[k] = True
                if "Condition Immunities" in i[0]:
                    for k in conditionImmunities:
                        if k.lower() in i[0]:
                            conditionImmunities[k] = True
                        
                    
        if "Tiny" in data[2][0]:
            size = "Tiny"
        elif "Small" in data[2][0]:
            size = "Small"
        elif "Medium" in data[2][0]:
            size = "Medium"
        elif "Large" in data[2][0]:
            size = "Large"
        elif "Huge" in data[2][0]:
            size = "Huge"
        elif "Gargantuan" in data[2][0]:
            size = "Gargantuan"
            
        print("Name:",name)
        print("Size:",size)
        print("AC:",armorClass)
        print("HP:",hitPoints)
        print("Speed:",speed)
        print("Stats:",stats)
        print("Saving Throws:",savingThrows)
        print("Skills:",skills)
        print("Resistances:", damageResistances,sep='\n')
        print("Vulnerabilities:", damageVulnerabilities,sep='\n')
        print("Immunities:", damageImmunities,sep='\n')
        print("Condition Immunities:", conditionImmunities,sep='\n')
        print("CR:",challengeRating)
        print("PB:",proficiencyBonus)
        print("")
        
        for header in headersOfInterest:
            if(header.string == "Legendary Actions"):
                instances = soup(text=re.compile("take \d legendary actions"))
                match = re.search(r'take \d legendary actions', instances[0])
                legendaryAct = match.group()[5]
            elif(header.string == "Mythic Actions"):
                mythic = True
            elif(header.string == "Regional Effects"):
                regional = True
            elif(header.string == "Lair Actions"):
                lair = True
            elif(header.string == "Bonus Actions"):
                bonus = True
            elif(header.string == "Reactions"):
                reaction = True
            else:
                action = True
        currentIndexOfInterest = 0
        if(len(placesOfInterest)-len(headersOfInterest)>0):
            featureList = placesOfInterest[currentIndexOfInterest]
            featureList = featureList.find_all("span",{'class': ["rd__h","rd__h--3"]})
            for featureOriginal in featureList:
                featureHTML = featureOriginal.find("span",{'class':'entry-title-inner'})
                feature = featureHTML.string
                if(feature == None):
                    featureHTML.find(True).replaceWith('')
                    featureHTML = featureOriginal.find("span",{'class':'entry-title-inner'}).contents[0]
                    feature = featureHTML.string
                feature = re.sub("[.]", "", feature)
                feature = re.sub(" \(", "_(", feature)
                feature = feature.replace("\\","")
                featureSplit = feature.split("_")
                if(featureSplit[0]=="Legendary Resistance"):
                    legendaryRes = featureSplit[1][1]
                else:
                    features.append(featureSplit[0])
                if(featureSplit[0]=="Spellcasting"):
                    spellCaster = True
            currentIndexOfInterest += 1

        if(action):
            actionList = placesOfInterest[currentIndexOfInterest]
            actionList = actionList.find_all("div",{'class': ['rd__b',"rd__b--3"]})
            actionNames = []
            for i, actionOriginal in enumerate(actionList):
                actionEntry = {}
                #Get action name
                actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'})
                actionName = actionNameHTML.string
                if(actionName == None):
                    actionNameHTML.find(True).replaceWith('')
                    actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'}).contents[0]
                    actionName = actionNameHTML.string
                actionName = re.sub("[.]", "", actionName)
                actionName = re.sub(" \(", "_(", actionName)
                actionName = actionName.replace("\\","")
                actionSplit = actionName.split("_")
                actionName = actionSplit[0]
                if(actionName == "Spellcasting"):
                    spellCaster = True
                elif(actionName == "Multiattack"):
                    multiAttack = True
                    continue
                actionNames.append(actionName)

                actionEntry["name"] = actionName
                
                toHit = actionOriginal.find("span",{"class":"rd__roller--roll-prof-bonus"})
                if(toHit != None):
                    actionEntry['toHit'] = toHit.string
                
                
                importantStats = actionOriginal.find_all(has_dice)
                damageIncrement = 0
                for stat in importantStats:
                    if '"subtype":"damage"' in str(stat) or 'subType&quot;:&quot;damage' in str(stat):
                        damageFormula = stat.string.replace(" ","")
                        diceRolls = re.search(r"[\d]+d[\d]+",damageFormula).group()
                        try:
                            modifier = re.search(r"[+-]\d+",damageFormula).group()
                        except:
                            modifier = 0
                        
                        PB = 0
                        if('PB' in damageFormula):
                            #FIX THIS BEFORE CALLING IT GOOD
                            pass
                        
                        numDice,diceType = diceRolls.split("d")
                        averagePerDie = (1 + int(diceType))/2
                        average = int(averagePerDie * int(numDice))
                        damageTotal = average + int(modifier) + PB

                        damageIncrement += 1
                        actionEntry['damage'+str(damageIncrement)]=damageTotal
                
                typeIncrement = 0
                dtypes = re.findall(r'\) [\S]+ damage',str(actionOriginal))
                for dtype in dtypes:
                    if(dtype == "half" or dtype == "full"):
                        continue
                    typeIncrement += 1
                    actionEntry['dtype' + str(typeIncrement)] = dtype.split(" ")[1]
                
                dc = actionOriginal.find("span",{"class":"rd__dc--rollable-text"})
                if(dc != None and actionName != "Spellcasting"):
                    actionEntry['DC'] = dc.string
                    
                appliedConditions = re.findall(conditionsRegex,str(actionOriginal))
                abilityConditions = []
                for appliedCondition in appliedConditions:
                    if(appliedCondition not in abilityConditions):
                        abilityConditions.append(appliedCondition)
                actionEntry["conditions"] = abilityConditions
                if("Spellcasting" in actionNames):
                    spellCaster = True
                else:
                    actions.append(actionEntry)
                
            if(multiAttack):
                multiAttackText = actionList[0]
                multiAttackText = multiAttackText.find("p").string
                match = re.search(r"level",str(multiAttackText))
                
                if(match==None):
                    regex = "one|two|three|four|five|six|seven|each"
                    if("Spellcasting" in actionNames):
                        actionNames.pop(actionNames.index("Spellcasting"))

                    for actionName in actionNames:
                        if " " in actionName:
                            newActionNameSplit = actionName.split(" ")
                            regexPiece = ""
                            for part in newActionNameSplit:
                                regexPiece += "[" + part[0].lower() + part[0].upper() + "]" + part[1:]
                                if(newActionNameSplit.index(part)<len(newActionNameSplit)-1):
                                    regexPiece += " "
                        else:
                            regexPiece = "[" + actionName[0].lower() + actionName[0].upper() + "]" + actionName[1:]
                        regex += "|"+ regexPiece
                    matches = re.findall(re.compile(regex),str(multiAttackText))
                    matches = [x.lower() for x in matches]
#                     print(name,matches,regex,actionNames)
                    multiattackArr = {'name':"Multiattack"}
    #                 print(name, matches)
                    if("each" in matches):
                        indexMatch = matches.index("each")
                        numTimes = 1
                        try:
                            if(matches[indexMatch-1] in list(num_dict.keys())):
                                numTimes = int(num_dict[matches[indexMatch-1]])
                        except:
                            pass
                        for actionName in actionNames:
                            multiattackArr[actionName] = numTimes

                    else:
                        for actionName in actionNames:
                            try:
                                indexMatch = matches.index(actionName.lower())
                                
                                if(matches[indexMatch-1] in list(nums_dict.keys())):
                                    multiattackArr[actionName] = int(nums_dict[matches[indexMatch-1]])
                                else:
                                    multiattackArr[actionName] = 1
                            except:
                                pass
    #                 print(name,multiattackArr)
                    if len(multiattackArr.keys())==1:
                        multiattackArr["complicated"] = True
                    else:
                        multiattackArr["complicated"] = False
                    actions.append(multiattackArr)
                    
                    
                
            currentIndexOfInterest += 1

            
        if(bonus):
            actionList = placesOfInterest[currentIndexOfInterest]
            actionList = actionList.find_all("div",{'class': ['rd__b',"rd__b--3"]})
            actionNames = []
            for i, actionOriginal in enumerate(actionList):
                actionEntry = {}
                #Get action name
                actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'})
                actionName = actionNameHTML.string
                if(actionName == None):
                    actionNameHTML.find(True).replaceWith('')
                    actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'}).contents[0]
                    actionName = actionNameHTML.string
                actionName = re.sub("[.]", "", actionName)
                actionName = re.sub(" \(", "_(", actionName)
                actionName = actionName.replace("\\","")
                actionSplit = actionName.split("_")
                actionName = actionSplit[0]
                if(actionName == "Spellcasting"):
                    spellCaster = True
                elif(actionName == "Multiattack"):
                    multiAttack = True
                    continue
                actionNames.append(actionName)

                actionEntry["name"] = actionName
                
                toHit = actionOriginal.find("span",{"class":"rd__roller--roll-prof-bonus"})
                if(toHit != None):
                    actionEntry['toHit'] = toHit.string
                
                
                importantStats = actionOriginal.find_all(has_dice)
                damageIncrement = 0
                for stat in importantStats:
                    if '"subtype":"damage"' in str(stat) or 'subType&quot;:&quot;damage' in str(stat):
                        damageFormula = stat.string.replace(" ","")
                        diceRolls = re.search(r"[\d]+d[\d]+",damageFormula).group()
                        try:
                            modifier = re.search(r"[+-]\d+",damageFormula).group()
                        except:
                            modifier = 0
                        
                        PB = 0
                        if('PB' in damageFormula):
                            #FIX THIS BEFORE CALLING IT GOOD
                            pass
                        
                        numDice,diceType = diceRolls.split("d")
                        averagePerDie = (1 + int(diceType))/2
                        average = int(averagePerDie * int(numDice))
                        damageTotal = average + int(modifier) + PB

                        damageIncrement += 1
                        actionEntry['damage'+str(damageIncrement)]=damageTotal
                
                typeIncrement = 0
                dtypes = re.findall(r'\) [\S]+ damage',str(actionOriginal))
                for dtype in dtypes:
                    if(dtype == "half" or dtype == "full"):
                        continue
                    typeIncrement += 1
                    actionEntry['dtype' + str(typeIncrement)] = dtype.split(" ")[1]
                
                dc = actionOriginal.find("span",{"class":"rd__dc--rollable-text"})
                if(dc != None and actionName != "Spellcasting"):
                    actionEntry['DC'] = dc.string
                    
                appliedConditions = re.findall(conditionsRegex,str(actionOriginal))
                abilityConditions = []
                for appliedCondition in appliedConditions:
                    if(appliedCondition not in abilityConditions):
                        abilityConditions.append(appliedCondition)
                actionEntry["conditions"] = abilityConditions
                if("Spellcasting" in actionNames):
                    spellCaster = True
                else:
                    bonuses.append(actionEntry)
            currentIndexOfInterest += 1
        if(reaction):
            actionList = placesOfInterest[currentIndexOfInterest]
            actionList = actionList.find_all("div",{'class': ['rd__b',"rd__b--3"]})
            actionNames = []
            for i, actionOriginal in enumerate(actionList):
                actionEntry = {}
                #Get action name
                actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'})
                actionName = actionNameHTML.string
                if(actionName == None):
                    actionNameHTML.find(True).replaceWith('')
                    actionNameHTML = actionOriginal.find("span",{'class':'entry-title-inner'}).contents[0]
                    actionName = actionNameHTML.string
                actionName = re.sub("[.]", "", actionName)
                actionName = re.sub(" \(", "_(", actionName)
                actionName = actionName.replace("\\","")
                actionSplit = actionName.split("_")
                actionName = actionSplit[0]
                if(actionName == "Spellcasting"):
                    spellCaster = True
                elif(actionName == "Multiattack"):
                    multiAttack = True
                    continue
                actionNames.append(actionName)

                actionEntry["name"] = actionName
                
                toHit = actionOriginal.find("span",{"class":"rd__roller--roll-prof-bonus"})
                if(toHit != None):
                    actionEntry['toHit'] = toHit.string
                
                
                importantStats = actionOriginal.find_all(has_dice)
                damageIncrement = 0
                for stat in importantStats:
                    if '"subtype":"damage"' in str(stat) or 'subType&quot;:&quot;damage' in str(stat):
                        damageFormula = stat.string.replace(" ","")
                        diceRolls = re.search(r"[\d]+d[\d]+",damageFormula).group()
                        try:
                            modifier = re.search(r"[+-]\d+",damageFormula).group()
                        except:
                            modifier = 0
                        
                       
                        
                        numDice,diceType = diceRolls.split("d")
                        averagePerDie = (1 + int(diceType))/2
                        average = int(averagePerDie * int(numDice))
                        
                        if('PB' in damageFormula):
                            damageTotal = average + int(modifier) + PB
                        else:
                            damageTotal = average + int(modifier)

                        damageIncrement += 1
                        actionEntry['damage'+str(damageIncrement)]=damageTotal
                
                typeIncrement = 0
                dtypes = re.findall(r'\) [\S]+ damage',str(actionOriginal))
                for dtype in dtypes:
                    if(dtype == "half" or dtype == "full"):
                        continue
                    typeIncrement += 1
                    actionEntry['dtype' + str(typeIncrement)] = dtype.split(" ")[1]
                
                dc = actionOriginal.find("span",{"class":"rd__dc--rollable-text"})
                if(dc != None and actionName != "Spellcasting"):
                    actionEntry['DC'] = dc.string
                    
                appliedConditions = re.findall(conditionsRegex,str(actionOriginal))
                abilityConditions = []
                for appliedCondition in appliedConditions:
                    if(appliedCondition not in abilityConditions):
                        abilityConditions.append(appliedCondition)
                actionEntry["conditions"] = abilityConditions
                if("Spellcasting" in actionNames):
                    spellCaster = True
                else:
                    reactions.append(actionEntry)
            currentIndexOfInterest += 1
            
        #We don't care about gathering the info in these, though we leave these here just in case
        #we care about trying to access the associated elements of information at a later date
        if(legendaryAct!=0):
            currentIndexOfInterest += 1
        if(mythic):
            currentIndexOfInterest += 1
        if(lair):
            currentIndexOfInterest += 1
        if(regional):
            currentIndexOfInterest += 1
        if(spellCaster):
            casters += 1


Name: Aarakocra Simulacrum
Size: Medium
AC: 12
HP: 6
Speed: {'walk': '20', 'climb': 0, 'fly': '50', 'swim': 0, 'burrow': 0}
Stats: {'strength': '10', 'dexterity': '14', 'constitution': '10', 'intelligence': '11', 'wisdom': '12', 'charisma': '11'}
Saving Throws: {'strength': 0, 'dexterity': 2, 'constitution': 0, 'intelligence': 0, 'wisdom': 1, 'charisma': 0}
Skills: {'Athletics': 0, 'Acrobatics': 2, 'Sleight of Hand,': 0, 'Stealth': 2, 'Arcana': 0, 'History': 0, 'Investigation': 0, 'Nature': 0, 'Religion': 0, 'Animal Handling': 1, 'Insight': 1, 'Medicine': 1, 'Perception': '5', 'Survival': 1, 'Deception': 0, 'Intimidation': 0, 'Performance': 0, 'Persuasion': 0, 'Sleight of Hand': 2}
Resistances:
{'Acid': False, 'Bludgeoning': False, 'Cold': False, 'Fire': False, 'Force': False, 'Lightning': False, 'Necrotic': False, 'Piercing': False, 'Poison': False, 'Psychic': False, 'Radiant': False, 'Slashing': False, 'Thunder': False}
Vulnerabilities:
{'Acid': False, 'Bludgeoning': False, 'Cold': F

Name: Aboleth Spawn
Size: Medium
AC: 14
HP: 93
Speed: {'walk': '30', 'climb': 0, 'fly': 0, 'swim': '30', 'burrow': 0}
Stats: {'strength': '19', 'dexterity': '11', 'constitution': '18', 'intelligence': '7', 'wisdom': '12', 'charisma': '9'}
Saving Throws: {'strength': 4, 'dexterity': 0, 'constitution': 4, 'intelligence': -2, 'wisdom': 1, 'charisma': -1}
Skills: {'Athletics': 4, 'Acrobatics': 0, 'Sleight of Hand,': 0, 'Stealth': 0, 'Arcana': -2, 'History': -2, 'Investigation': -2, 'Nature': -2, 'Religion': -2, 'Animal Handling': 1, 'Insight': 1, 'Medicine': 1, 'Perception': '4', 'Survival': 1, 'Deception': -1, 'Intimidation': -1, 'Performance': -1, 'Persuasion': -1, 'Sleight of Hand': 0}
Resistances:
{'Acid': False, 'Bludgeoning': False, 'Cold': False, 'Fire': False, 'Force': False, 'Lightning': False, 'Necrotic': False, 'Piercing': False, 'Poison': False, 'Psychic': True, 'Radiant': False, 'Slashing': False, 'Thunder': False}
Vulnerabilities:
{'Acid': False, 'Bludgeoning': False, 'Cold':

Name: Adult Blue Dragon
Size: Huge
AC: 19
HP: 225
Speed: {'walk': '40', 'climb': 0, 'fly': '80', 'swim': 0, 'burrow': '30'}
Stats: {'strength': '25', 'dexterity': '10', 'constitution': '23', 'intelligence': '16', 'wisdom': '15', 'charisma': '19'}
Saving Throws: {'strength': 7, 'dexterity': '5', 'constitution': '11', 'intelligence': 3, 'wisdom': '7', 'charisma': '9'}
Skills: {'Athletics': 7, 'Acrobatics': 0, 'Sleight of Hand,': 0, 'Stealth': '12', 'Arcana': 3, 'History': 3, 'Investigation': 3, 'Nature': 3, 'Religion': 3, 'Animal Handling': 2, 'Insight': '1', 'Medicine': 2, 'Perception': '12', 'Survival': 2, 'Deception': 4, 'Intimidation': 4, 'Performance': 4, 'Persuasion': 4, 'Sleight of Hand': 0}
Resistances:
{'Acid': False, 'Bludgeoning': False, 'Cold': False, 'Fire': False, 'Force': False, 'Lightning': False, 'Necrotic': False, 'Piercing': False, 'Poison': False, 'Psychic': False, 'Radiant': False, 'Slashing': False, 'Thunder': False}
Vulnerabilities:
{'Acid': False, 'Bludgeoning': F

Name: Adult Gold Dragon
Size: Huge
AC: 19
HP: 256
Speed: {'walk': '40', 'climb': 0, 'fly': '80', 'swim': '40', 'burrow': 0}
Stats: {'strength': '27', 'dexterity': '14', 'constitution': '25', 'intelligence': '16', 'wisdom': '15', 'charisma': '24'}
Saving Throws: {'strength': 8, 'dexterity': '8', 'constitution': '13', 'intelligence': 3, 'wisdom': '8', 'charisma': '13'}
Skills: {'Athletics': 8, 'Acrobatics': 2, 'Sleight of Hand,': 0, 'Stealth': '15', 'Arcana': 3, 'History': 3, 'Investigation': 3, 'Nature': 3, 'Religion': 3, 'Animal Handling': 2, 'Insight': '1', 'Medicine': 2, 'Perception': '15', 'Survival': 2, 'Deception': 7, 'Intimidation': 7, 'Performance': 7, 'Persuasion': '13', 'Sleight of Hand': 2}
Resistances:
{'Acid': False, 'Bludgeoning': False, 'Cold': False, 'Fire': False, 'Force': False, 'Lightning': False, 'Necrotic': False, 'Piercing': False, 'Poison': False, 'Psychic': False, 'Radiant': False, 'Slashing': False, 'Thunder': False}
Vulnerabilities:
{'Acid': False, 'Bludgeoning

Name: Adult Sapphire Dragon
Size: Huge
AC: 18
HP: 225
Speed: {'walk': '40', 'climb': '40', 'fly': '80', 'swim': 0, 'burrow': '30'}
Stats: {'strength': '23', 'dexterity': '14', 'constitution': '22', 'intelligence': '18', 'wisdom': '17', 'charisma': '18'}
Saving Throws: {'strength': 6, 'dexterity': '7', 'constitution': '11', 'intelligence': 4, 'wisdom': '8', 'charisma': '9'}
Skills: {'Athletics': 6, 'Acrobatics': 2, 'Sleight of Hand,': 0, 'Stealth': '12', 'Arcana': 4, 'History': '9', 'Investigation': 4, 'Nature': 4, 'Religion': 4, 'Animal Handling': 3, 'Insight': '1', 'Medicine': 3, 'Perception': '12', 'Survival': 3, 'Deception': 4, 'Intimidation': 4, 'Performance': 4, 'Persuasion': '14', 'Sleight of Hand': 2}
Resistances:
{'Acid': False, 'Bludgeoning': False, 'Cold': False, 'Fire': False, 'Force': False, 'Lightning': True, 'Necrotic': False, 'Piercing': False, 'Poison': False, 'Psychic': False, 'Radiant': False, 'Slashing': False, 'Thunder': True}
Vulnerabilities:
{'Acid': False, 'Bludg

...

Name: Archon of the Triumvirate
Size: Medium
AC: 18
HP: 144
Speed: {'walk': '30', 'climb': 0, 'fly': 0, 'swim': 0, 'burrow': 0}
Stats: {'strength': '20', 'dexterity': '15', 'constitution': '19', 'intelligence': '15', 'wisdom': '21', 'charisma': '18'}
Saving Throws: {'strength': 5, 'dexterity': 2, 'constitution': '9', 'intelligence': 2, 'wisdom': '10', 'charisma': '9'}
Skills: {'Athletics': 5, 'Acrobatics': 2, 'Sleight of Hand,': 0, 'Stealth': 2, 'Arcana': 2, 'History': 2, 'Investigation': 2, 'Nature': 2, 'Religion': 2, 'Animal Handling': 5, 'Insight': '10', 'Medicine': 5, 'Perception': '10', 'Survival': 5, 'Deception': 4, 'Intimidation': 4, 'Performance': 4, 'Persuasion': 4, 'Sleight of Hand': 2}
Resistances:
{'Acid': False, 'Bludgeoning': True, 'Cold': False, 'Fire': False, 'Force': False, 'Lightning': False, 'Necrotic': False, 'Piercing': True, 'Poison': False, 'Psychic': False, 'Radiant': False, 'Slashing': True, 'Thunder': False}
Vulnerabilities:
{'Acid': False, 'Bludgeoning': Fals

Name: Ashen Knight
Size: Medium
AC: 18
HP: 52
Speed: {'walk': '30', 'climb': 0, 'fly': 0, 'swim': 0, 'burrow': 0}
Stats: {'strength': '16', 'dexterity': '11', 'constitution': '14', 'intelligence': '11', 'wisdom': '11', 'charisma': '15'}
Saving Throws: {'strength': 3, 'dexterity': 0, 'constitution': '4', 'intelligence': 0, 'wisdom': '2', 'charisma': 2}
Skills: {'Athletics': 3, 'Acrobatics': 0, 'Sleight of Hand,': 0, 'Stealth': 0, 'Arcana': 0, 'History': 0, 'Investigation': 0, 'Nature': 0, 'Religion': 0, 'Animal Handling': 0, 'Insight': 0, 'Medicine': 0, 'Perception': 0, 'Survival': 0, 'Deception': 2, 'Intimidation': 2, 'Performance': 2, 'Persuasion': 2, 'Sleight of Hand': 0}
Resistances:
{'Acid': False, 'Bludgeoning': False, 'Cold': False, 'Fire': False, 'Force': False, 'Lightning': False, 'Necrotic': False, 'Piercing': False, 'Poison': False, 'Psychic': False, 'Radiant': False, 'Slashing': False, 'Thunder': False}
Vulnerabilities:
{'Acid': False, 'Bludgeoning': False, 'Cold': False, 'F

Name: Astral Elf Aristocrat
Size: Medium
AC: 16
HP: 103
Speed: {'walk': '30', 'climb': 0, 'fly': 0, 'swim': 0, 'burrow': 0}
Stats: {'strength': '11', 'dexterity': '14', 'constitution': '10', 'intelligence': '21', 'wisdom': '18', 'charisma': '18'}
Saving Throws: {'strength': 0, 'dexterity': 2, 'constitution': 0, 'intelligence': '8', 'wisdom': '7', 'charisma': '7'}
Skills: {'Athletics': 0, 'Acrobatics': 2, 'Sleight of Hand,': 0, 'Stealth': 2, 'Arcana': '8', 'History': 5, 'Investigation': 5, 'Nature': 5, 'Religion': 5, 'Animal Handling': 4, 'Insight': '7', 'Medicine': 4, 'Perception': 4, 'Survival': 4, 'Deception': '7', 'Intimidation': 4, 'Performance': 4, 'Persuasion': '7', 'Sleight of Hand': 2}
Resistances:
{'Acid': False, 'Bludgeoning': False, 'Cold': False, 'Fire': False, 'Force': False, 'Lightning': False, 'Necrotic': False, 'Piercing': False, 'Poison': False, 'Psychic': False, 'Radiant': False, 'Slashing': False, 'Thunder': False}
Vulnerabilities:
{'Acid': False, 'Bludgeoning': Fals

Name: Aurumvorax Den Leader
Size: Medium
AC: 16
HP: 52
Speed: {'walk': '40', 'climb': 0, 'fly': 0, 'swim': 0, 'burrow': '20'}
Stats: {'strength': '18', 'dexterity': '14', 'constitution': '14', 'intelligence': '3', 'wisdom': '13', 'charisma': '8'}
Saving Throws: {'strength': '6', 'dexterity': 2, 'constitution': '4', 'intelligence': -4, 'wisdom': 1, 'charisma': -1}
Skills: {'Athletics': 4, 'Acrobatics': 2, 'Sleight of Hand,': 0, 'Stealth': '4', 'Arcana': -4, 'History': -4, 'Investigation': -4, 'Nature': -4, 'Religion': -4, 'Animal Handling': 1, 'Insight': 1, 'Medicine': 1, 'Perception': '3', 'Survival': 1, 'Deception': -1, 'Intimidation': -1, 'Performance': -1, 'Persuasion': -1, 'Sleight of Hand': 2}
Resistances:
{'Acid': False, 'Bludgeoning': False, 'Cold': False, 'Fire': False, 'Force': False, 'Lightning': False, 'Necrotic': False, 'Piercing': False, 'Poison': False, 'Psychic': False, 'Radiant': False, 'Slashing': False, 'Thunder': False}
Vulnerabilities:
{'Acid': False, 'Bludgeoning':

AttributeError: 'NoneType' object has no attribute 'group'

In [122]:
print(actions,bonuses,reactions)

[{'name': 'Pseudopod', 'toHit': '+13', 'damage1': 15, 'damage2': 9, 'dtype1': 'force', 'dtype2': 'poison', 'conditions': []}, {'name': 'Multiattack', 'Pseudopod': 3, 'complicated': False}] [{'name': 'Infestation Spores', 'DC': '19', 'conditions': []}, {'name': 'Mind Control Spores', 'DC': '19', 'conditions': ['charmed']}] [{'name': 'Protective Thrall', 'conditions': ['charmed']}]
